In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read and merge Census

In [2]:
data_path = "../data/data_2017/"
acs1 = pd.read_csv(data_path + 'ACSDP5Y2017.DP03_data_with_overlays_2022-03-17T161301.csv', na_values=['(X)']) #, encoding='cp1252'
acs2 = pd.read_csv(data_path + 'ACSDP5Y2017.DP05_data_with_overlays_2022-03-01T200530.csv', na_values=['(X)']) 
acs3 = pd.read_csv(data_path + 'ACSST5Y2017.S1701_data_with_overlays_2022-04-06T222046.csv', na_values=['(X)'])

#DP03_0095E - health care coverage

def remove_cols(df):
    
    #save county
    county = df['GEO_ID']
    county = county.str[-5:]
    county = county.astype(str) 
    county = county.loc[1:]
    
    #find the bad columns
    cols = list(df.columns)
    trim_cols1 = []
    for col in cols:
        income_col = 'C01_' in col and 'E' in col
        if 'PE' in col or 'DP03_0095E' in col or income_col:
            trim_cols1.append(col)

    df = df[trim_cols1]
    df = df.loc[1:]
    df.fillna(0, inplace = True)
    
    
    #filter out bad columns
    df['County'] = county
    
    return df


dfs = []

for i in [acs1,acs2,acs3]:
    dfs.append(remove_cols(i))

for i in dfs:
    print(i.shape)

(3220, 139)
(3220, 90)
(3220, 62)


In [3]:
from functools import partial, reduce
[acs1, acs2,acs3] = dfs
merge = partial(pd.merge, on=['County'], how='outer')
all_acs = reduce(merge, dfs)
all_acs

,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,...,S1701_C01_052E,S1701_C01_053E,S1701_C01_054E,S1701_C01_055E,S1701_C01_056E,S1701_C01_057E,S1701_C01_058E,S1701_C01_059E,S1701_C01_060E,S1701_C01_061E
0,42974,60.3,59.2,56.1,3.1,1.1,39.7,25436,5.2,22202,...,954,757,1294,1406,1294,1257,5908,3236,978,3559
1,163759,58.0,57.9,54.7,3.2,0.1,42.0,94742,5.5,84553,...,4604,4039,5332,6831,5515,5557,6048,12706,6877,15035
2,21241,47.7,47.7,41.8,5.9,0.0,52.3,10139,12.4,9853,...,601,404,632,690,648,713,5896,1214,657,1979
3,18415,48.3,48.3,44.4,4.0,0.0,51.7,8899,8.2,8462,...,293,328,478,346,420,353,7601,712,272,1396
4,45924,49.0,48.9,46.6,2.4,0.1,51.0,22471,4.9,23323,...,928,573,1126,1452,1262,1212,6690,2193,864,3839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,44583,38.4,38.4,31.9,6.4,0.0,61.6,17098,16.8,23602,...,640,712,1060,1214,1427,1221,7643,1369,511,4632
3216,7136,47.0,47.0,41.0,6.0,0.0,53.0,3355,12.8,3573,...,53,467,150,269,370,338,7133,509,304,858
3217,18999,48.2,48.1,36.2,11.9,0.1,51.8,9141,24.8,9885,...,413,321,442,567,610,286,6768,612,523,1622
3218,28515,37.1,37.0,27.6,9.4,0.1,62.9,10563,25.4,14820,...,462,761,860,1425,1192,754,7830,902,430,4432


In [4]:
all_acs.to_csv('../data/processed_data/county_characteristics_2017.csv', index=False)